<a href="https://colab.research.google.com/github/alen-ho/EnvDatSci/blob/main/finished_py_script/automaticdatadownloads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic Data Downloads
Satellite images and outputs from global earth systems models can be very large files. If we're dealing with time series, large spatial areas, or multivariate model outputs, we can quickly be moving into data volumes that exceed the memory and storage capacity of personal computers. To access these types of global data, we are interfacing with online databases. Today's lesson is intended to give you the tools to programmatically access online databases. These tools will enable you to use your personal computer to convert these large datasets into analysis-ready data for your research project. Specifically, today we'll learn to:

1. Interpret directory structure of ftp and http addresses.
2. Create a project directory on your local machine.
3. Configure a .gitignore file to ignore raw data.
4. Use the command line to download files from the internet.

If there's time, we'll break into groups based on research interest and start utilizing APIs to search datasets on public geospatial data repositories that match the location and time period of your study area.

In [ ]:
import pandas as pd
from IPython.display import HTML
import os
import urllib.request

## G is for *Generalizable*
When we're making measurements of an earth system process, we often care deeply about how well uur experimental results apply to other times/places. Since it is often too expensive or two difficult to collect in-situ samples of our earth systems process at all the times and locations that matter, environmental data science allows us to use statistical models to leverage globally available observations to improve the generalizability of our system. These models will generalize our inferences about our earth systems process in one of three ways:

1. *Prediction*: can our model allow us to generalize our observations to out-of-sample times and locations? For example: will my model linking air temperature to green-up time from my experimental forest accurately apply to a forest 200 miles away?
2. *Interpolation*: can our model allow us to "fill in the gaps" in our spatial/temporal sampling schele? For example: do my measurements of precipitation for my two precipitation gage locations accurately represent the total precipitation that fell in my watershed?
3. *Diagnosis*: can our model help us to interpret what processes are either drivers of or covariates with our earth systems process, allowing us to improve our physical understanding of trends and variability in that system: for example: is air temperature or precipitation a more important driver of current cropping system productivity, and how might this impact cropping system function under climate change?

### These global observations are often publically available to researchers on online geodatabases.
For example:
 - NASA: https://earthdata.nasa.gov/
 - USGS: https://earthexplorer.usgs.gov/
 - NOAA: https://psl.noaa.gov/data/gridded/
 - Google Earth Engine: https://developers.google.com/earth-engine/datasets
 - NY State: https://cugir.library.cornell.edu/


## R is for *Reproducible*
Since the raw data for our generalizable analysis is globally available, programmatically accessing our data gives us an important added benefit: we can design our version controlled, collaborative project repositories so they directly interface with these public geodatabases. That way, anyone who wants to can access the raw data required to reproduce our analytic workflow.

A reminder on why reproducible science is so important:

In [ ]:
HTML('<iframe width="930" height="523" src="https://www.youtube.com/embed/NGFO0kdbZmk", frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.10/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### Project Repository
Your project repository is where you store all of the elements of your data science workflow. At it's core, it should have folders for raw data, processed data, code, outputs, and images. A good project repository is.

1. Human readable: use directory names that are easy to understand, includes a highly detailed README file that explains what's in each folder, how to sequence inputs and outputs to code files, and how to cite the repository.
2. Machine readable - avoid funky characters OR SPACES.
3. Supportive of sorting - If you have a list of input files, it’s nice to be able to sort them to quickly see what’s there and find what you need.

You should also take extra steps to preserve raw data so it’s not modified. More on this later.

We're going to create a new repository for your class project. The os package (os stands for **O**perating **S**ystem) allows you to manipulate files on your computer. Ask it what it does:

In [ ]:
?os

In [ ]:
#For example, this command is the equivalent of ls in terminal:
os.getcwd()

'/content/C:\\EnvDatSci\\project'

In [ ]:
# RUN THIS CELL IF YOU ARE ON A PC!

#this command is the equivalent of:
# mkdir C:/EnvDatSci/project
os.mkdir('C:\\EnvDatSci\\project')

#this command is the equivalent of:
# cd C:/EnvDatSci/project
os.chdir('C:\\EnvDatSci\\project')

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
# RUN THIS CELL IF YOU ARE ON GOOGLE COLAB!
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/env_data/project")
#this command is the equivalend of:
#mkdir content/drive/MyDrive/EnvDatSci/project

# this command is the equivalent of:
#cd  content/drive/MyDrive/EnvDatSci/project


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### TASK 1: enter a command in the below cell to check and make sure you're in your project directory:

In [25]:
#Task 1:
os.getcwd()

'/content/drive/MyDrive/env_data/project'

### TASK 2: populate your project directory with appropriate files
Read Chapter 4.1 of the textbook: https://www.earthdatascience.org/courses/earth-analytics/document-your-science/file-organization-101/

Using os commands, populate your project directory with subfolders.

Print your directory to the screen (hint: see Task 1)

In [39]:
#Task 2:
path = "/content/drive/MyDrive/env_data/project/test_folder"
os.makedirs(path, exist_ok=True)

In [42]:
path_sub = "/content/drive/MyDrive/env_data/project/test_folder/test_subfolder"
os.makedirs(path_sub, exist_ok=True)

In [43]:
os.getcwd()

'/content/drive/MyDrive/env_data/project'

### TASK 3: change the current working directory to your the folder where you intend to store raw data.

In [44]:
#Task 3:
os.getcwd()

'/content/drive/MyDrive/env_data/project'

In [45]:
os.chdir("/content/drive/MyDrive/env_data/project/test_folder/test_subfolder")

In [46]:
os.getcwd()

'/content/drive/MyDrive/env_data/project/test_folder/test_subfolder'

## Decoding the file structure of online geodatabases
Just like we can use code to find and access files on our local machine, we can use code to find and access files on public geodatabases. Since these geodatabases are version controlled, providing code that links to the online files helps prevent us from making redundant copies of data on the internet. Programatically accessing public geodatabases requires that we understand how the database itself has been organized.

 - Click on the following link to the National Oceanic and Atmospheric Association databse website: https://psl.noaa.gov/data/gridded/

 - Navigate to the "NCEP/NCAR Reanalysis dataset"
 - Of the seven sections they've divided data into, click on "Surface"
 - Under "Air Temperature: Daily", click "See list"
 - Under "Surface", click "See list"

### TASK 4: Right click on the first link in the list, and select "copy link". Paste that link address below:
https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.1948.nc

##### Task 4: double click on this markdown cell to add text

https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.1948.nc

### Tasking your computer to download files
Our goal is to write a script that can download files, extract a relevant subset of information from the files, and then delete the files. The first part of this task to to learn the filenames that we want to download.

In the link above, we can break the filepath down into substrings, using basic text commands:

In [47]:
http_dir = "https://downloads.psl.noaa.gov/Datasets/"
dataset = "ncep.reanalysis.dailyavgs"
lev_type = "surface"
variable = "air.sig995."
time = "2010"
file_type = ".nc"
filepaths= http_dir + dataset + "/" + lev_type + "/" + variable + time + file_type
print(filepaths)

https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.2010.nc


What happens if you click on that link? You can also have python download the file for you using the <urllib.request.urlretrieve> function:

In [48]:
#what does this function do and how do we use it?
?urllib.request.urlretrieve

In [61]:
print(url)

https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.2010.nc


In [62]:
print(filename)

air.sig995.2010.nc


In [49]:
url = filepaths
filename = variable + time + file_type
urllib.request.urlretrieve(url, filename)
print(url, filename)

https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.2010.nc air.sig995.2010.nc


In [50]:
#what happens?
os.listdir()

['air.sig995.2010.nc']

We can infer patterns from the database itself and generate the names of multiple files. For example, if we need five years of daily air temperature data:

In [51]:
time =pd.Series(list(range(1965,1970)))
time = time.apply(str)
filepaths= http_dir + dataset + "/" + lev_type + "/" + variable + time + file_type
print(filepaths)

0    https://downloads.psl.noaa.gov/Datasets/ncep.r...
1    https://downloads.psl.noaa.gov/Datasets/ncep.r...
2    https://downloads.psl.noaa.gov/Datasets/ncep.r...
3    https://downloads.psl.noaa.gov/Datasets/ncep.r...
4    https://downloads.psl.noaa.gov/Datasets/ncep.r...
dtype: object


### TASK 5: Write a "for" loop that downloads all five years worth of air temperature data into you working directory. Print the contents of your directory to the screen.

In [71]:
filenames = variable + time + file_type
print(filenames)

0    air.sig995.1965.nc
1    air.sig995.1966.nc
2    air.sig995.1967.nc
3    air.sig995.1968.nc
4    air.sig995.1969.nc
dtype: object


In [72]:
#Task 5
for file, filename in zip(filepaths, filenames):
    urllib.request.urlretrieve(file, filename)
    print(f"Downloaded {file} as {filename}")

Downloaded https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.1965.nc as air.sig995.1965.nc
Downloaded https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.1966.nc as air.sig995.1966.nc
Downloaded https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.1967.nc as air.sig995.1967.nc
Downloaded https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.1968.nc as air.sig995.1968.nc
Downloaded https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis.dailyavgs/surface/air.sig995.1969.nc as air.sig995.1969.nc


In [73]:
os.listdir()

['air.sig995.2010.nc',
 'air.sig995.1965.nc',
 'air.sig995.1966.nc',
 'air.sig995.1967.nc',
 'air.sig995.1968.nc',
 'air.sig995.1969.nc']